# Generating a list of Animeplanet usernames
* from recent webpages

In [ ]:
import os
import re

from filelock import FileLock

In [ ]:
PROXY_NUMBER = 0

In [ ]:
name = "get_recent_usernames"
source = "animeplanet"

In [ ]:
exec(open("WebEndpointBase.py").read())

In [ ]:
LOCK_FILE = "../../animeplanet_recent.lock"
LOCK = FileLock(LOCK_FILE, timeout=-1)

In [ ]:
def get_users(url, prefix):
    response = call_api(url)
    if response.status_code in [404]:
        logger.warning(f"Error {response} received when handling {url}")
        return {}
    if not response.ok:
        logger.warning(f"Error {response} received when handling {url}")
        return {}
    urls = re.findall(prefix + """[^"/#%?.'><]+""", response.text)
    return {x[len(prefix) :] for x in urls}

In [ ]:
def get_character_urls():
    urls = [
        (x, "/characters/")
        for x in [
            "https://www.anime-planet.com/characters/",
            "https://www.anime-planet.com/characters/top-loved",
            "https://www.anime-planet.com/characters/top-loved/today",
            "https://www.anime-planet.com/characters/top-loved/week",
            "https://www.anime-planet.com/characters/top-hated",
            "https://www.anime-planet.com/characters/top-hated/today",
            "https://www.anime-planet.com/characters/top-hated/week",
        ]
    ]
    users = set()
    for url, prefix in urls:
        users |= get_users(url, prefix)
    return [(f"https://www.anime-planet.com/characters/{x}", "/users/") for x in users]

In [ ]:
def get_frontpage_urls():
    return [
        (x, "/users/")
        for x in [
            "https://www.anime-planet.com/community/",
            "https://www.anime-planet.com/users/recent_user_reviews.php?mode=anime",
            "https://www.anime-planet.com/users/recent_user_reviews.php?mode=manga",
            "https://www.anime-planet.com/users/recent_recommendations.php?filter=anime&page=1",
            "https://www.anime-planet.com/users/recent_recommendations.php?filter=manga&page=1",
        ]
    ]

In [ ]:
def get_forum_urls():
    return [
        ("https://www.anime-planet.com/forum/", "members/"),
        ("https://www.anime-planet.com/forum/recent-activity/", "members/"),
        ("https://www.anime-planet.com/forum/members/", "members/"),
        ("https://www.anime-planet.com/forum/find-new/1/posts", "members/"),
        ("https://www.anime-planet.com/forum/find-new/1/profile-posts", "members/"),
        ("https://www.anime-planet.com/forum/online/?type=registered", "members/"),
    ]

In [ ]:
def reset_state():
    usernames = set()
    if os.path.exists("recent_usernames.txt"):
        with open("recent_usernames.txt") as f:
            usernames = {x.strip() for x in f.readlines() if x.strip()}
    return usernames

In [ ]:
while True:
    usernames = reset_state()
    for fn in [get_frontpage_urls, get_forum_urls, get_character_urls]:
        urls = fn()
        for url, prefix in urls:
            num_usernames = len(usernames)
            usernames |= get_users(url, prefix)
            if len(usernames) > num_usernames:
                with LOCK:
                    atomic_to_csv(sorted(list(usernames)), "recent_usernames.txt")
                    logger.info(f"Successfully written {len(usernames)} users")